<a href="https://colab.research.google.com/github/talktovishal/NLP-ML-Projects/blob/master/Project2_word2vec_Part2_PrepDataForCBOW_Frankenstien.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pwd
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

/content
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls "/content/drive/My Drive/Public Shared"
!cp "/content/drive/My Drive/Public Shared/Project-2-Word-Embeddings/normalizedSentences-frankenstein.pickle" "normalizedSentences.pickle"
#!cp "/content/drive/My Drive/Public Shared/Project-2-Word-Embeddings/mini-normalizedSentences.pickle" "normalizedSentences.pickle"

 Project-2-Word-Embeddings  'vishalc final certificate.pdf'


In [0]:
import pickle
import time

t = time.process_time()
# By choosing to open the file in mode wb, you are choosing to write in raw binary. There is no character encoding being applied.
# Thus to read this file, you should simply open in mode rb.
with open('normalizedSentences.pickle', 'rb') as f:
  normalizedSentences = pickle.load(f)

print(time.process_time() - t)


0.005084805000000081


In [0]:
len(normalizedSentences)


3427

In [0]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
normalizedSentences[345]

'it is even possible that the train of my ideas would never have received the fatal impulse that led to my ruin'

In [0]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

start_time = time.process_time()
#for stop words
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 

#wordsInSentenceWithStopWords = [word_tokenize(s) for s in normalizedSentences]
#remove stop words else you have a high degree of predicting the, of, a etc.
wordsInSentence = [tuple(w for w in word_tokenize(s) if not w in stop_words) for s in normalizedSentences]
print(time.process_time() - start_time)
#WHY is tuple needed in the expression above?
#https://dev.to/vmesel/what-the-heck-are-python-generator-expressions-and-list-comprehensions-55hh
#https://stackoverflow.com/questions/27968339/python-why-does-list-comprehension-produce-a-generator


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
0.5316614729999998


In [0]:
len(wordsInSentence)

wordsInSentence[0]


('frankenstein',
 'modern',
 'prometheus',
 'mary',
 'wollstonecraft',
 'godwin',
 'shelley',
 'letter',
 'st',
 'petersburgh',
 'dec',
 'th',
 'mrs',
 'saville',
 'england',
 'rejoice',
 'hear',
 'disaster',
 'accompanied',
 'commencement',
 'enterprise',
 'regarded',
 'evil',
 'forebodings')

In [0]:
#get the vocab
vocab = set()
for words in wordsInSentence:
  vocab.update(words)

len(vocab)  

7314

In [0]:
import random
for i, val in enumerate(random.sample(vocab, 10)):
  print(val)


foldings
neater
fettered
periods
liberal
waters
clasped
avenge
indulged
weeks


In [0]:
word_to_ix = {word: i for i, word in enumerate(vocab)}
ix_to_word = {i: word for i, word in enumerate(vocab)}

In [0]:
import numpy as np
data = []
for raw_text in wordsInSentence:
  for i in range(2, len(raw_text) - 2):
      #create numpy array instead of list as it helps later when converting into pytorch tensors
      context = np.array([
                 word_to_ix[raw_text[i - 2]], 
                 word_to_ix[raw_text[i - 1]],
                 word_to_ix[raw_text[i + 1]], 
                 word_to_ix[raw_text[i + 2]]
                 ])
      target = word_to_ix[raw_text[i]]
      data.append((context, target))


In [0]:
print(data[:5])  

[(array([2693, 3277, 5013, 5680]), 4122), (array([3277, 4122, 5680, 4033]), 5013), (array([4122, 5013, 4033, 1593]), 5680), (array([5013, 5680, 1593, 2149]), 4033), (array([5680, 4033, 2149, 4812]), 1593)]


In [0]:
import pandas as pd
cbow_data = pd.DataFrame(data, columns = ['Context', 'Target'])

In [0]:
cbow_data.head()


In [0]:
n = len(cbow_data)
train_proportion=0.9
val_proportion=0.05
test_proportion=0.05

#randomization.
cbow_data = cbow_data.sample(frac=1).reset_index(drop=True)


def get_split(row_num):
    if row_num <= n*train_proportion:
        return 'train'
    elif (row_num > n*train_proportion) and (row_num <= n*train_proportion + n*val_proportion):
        return 'val'
    else:
        return 'test'
        
cbow_data['split']= cbow_data.apply(lambda row: get_split(row.name), axis=1)

In [0]:
cbow_data.head()

,Context,Target,split
0,"[3774, 168, 1108, 5341]",4763,train
1,"[2491, 1115, 158, 2321]",2691,train
2,"[6284, 6712, 3939, 3213]",1698,train
3,"[4055, 3083, 4673, 6318]",5028,train
4,"[1178, 3528, 6436, 5558]",5200,train


In [0]:
"""
with open('cbow_data.pickle', 'wb') as f:
    pickle.dump(cbow_data, f)

with open('word_to_ix.pickle', 'wb') as f:
    pickle.dump(word_to_ix, f)

with open('vocab.pickle', 'wb') as f:
    pickle.dump(vocab, f)
"""    

In [0]:
len(cbow_data)

23307

In [0]:
training_data = cbow_data[cbow_data['split'] == 'train']
training_data.head()
#training_data.count()

,Context,Target,split
0,"[3774, 168, 1108, 5341]",4763,train
1,"[2491, 1115, 158, 2321]",2691,train
2,"[6284, 6712, 3939, 3213]",1698,train
3,"[4055, 3083, 4673, 6318]",5028,train
4,"[1178, 3528, 6436, 5558]",5200,train


In [0]:
#structure copied from https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim



In [0]:

#batch size == 1. VERY VERY SLOW
"""
def make_context_vector(context_ids):
    return torch.tensor(context_ids, dtype=torch.long)

class CBOW(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_layer = 128):
      super(CBOW, self).__init__()
      self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
      self.linear1 = nn.Linear(in_features=embedding_dim, out_features=hidden_layer)
      self.linear2 = nn.Linear(in_features=hidden_layer, out_features=vocab_size)


    def forward(self, inputs):
      x_embedded_sum = sum(self.embeddings(inputs)).view((1,-1))
      # https://stackoverflow.com/questions/42479902/how-does-the-view-method-work-in-pytorch
      # What is the meaning of parameter -1?
      # If there is any situation that you don't know how many rows you want but are sure of the number 
      # of columns, then you can specify this with a -1. (Note that you can extend this to tensors with 
      # more dimensions. Only one of the axis value can be -1). This is a way of telling the library: 
      # "give me a tensor that has these many columns and you compute the appropriate number of rows that 
      # is necessary to make this happen".      
      out1 = F.relu(self.linear1(x_embedded_sum))
      out2 = self.linear2(out1)
      log_probs = F.log_softmax(out2, dim=1)
      # axis : axis along which we want to calculate the sum value. Otherwise, it will consider arr to be 
      # flattened(works on all the axis). axis = 0 means along the column and axis = 1 means working along 
      # the row.
      return log_probs

    def get_word_emdedding(self, word):
      word = torch.LongTensor([word_to_ix[word]])
      return self.embeddings(word).view(1,-1)      


counter = 0
losses = []

EMBEDDING_DIM = 99
model = CBOW(vocab_size=len(vocab), embedding_dim=EMBEDDING_DIM)
loss_funtion = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)


for epoch in range(50):
  total_loss = 0
  #iterate over the rows in df -- note doing one at a time, not batches yet.
  for index, current_row in training_data.head(n=10000).iterrows():
    # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
    # into integer indices and wrap them in tensors)    
    context_vector = make_context_vector(current_row['Context'])
    model.zero_grad()
    log_probs = model(context_vector)
    loss = loss_funtion(log_probs, torch.tensor([current_row['Target']], dtype=torch.long))
    loss.backward()
    optimizer.step()
    # Get the Python number from a 1-element Tensor by calling tensor.item()
    total_loss += loss.item()
    counter += 1
    # if(counter % 1000 == 0):
    #   print(total_loss)
  print(f"{epoch}:{loss.item()}")
  losses.append(total_loss)

print(losses)
"""
    


'\ndef make_context_vector(context_ids):\n    return torch.tensor(context_ids, dtype=torch.long)\n\nclass CBOW(nn.Module):\n\n    def __init__(self, vocab_size, embedding_dim, hidden_layer = 128):\n      super(CBOW, self).__init__()\n      self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)\n      self.linear1 = nn.Linear(in_features=embedding_dim, out_features=hidden_layer)\n      self.linear2 = nn.Linear(in_features=hidden_layer, out_features=vocab_size)\n\n\n    def forward(self, inputs):\n      x_embedded_sum = sum(self.embeddings(inputs)).view((1,-1))\n      # https://stackoverflow.com/questions/42479902/how-does-the-view-method-work-in-pytorch\n      # What is the meaning of parameter -1?\n      # If there is any situation that you don\'t know how many rows you want but are sure of the number \n      # of columns, then you can specify this with a -1. (Note that you can extend this to tensors with \n      # more dimensions. Only one of the axis v

In [0]:
# preferred method: device agnostic tensor instantiation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print (device)

cpu


In [0]:
class CBOW_Batch(nn.Module):

    """
      x_0
      x_1                                     W_context                                     W_words
            ==> sum(x0... x3)   * [embedding_dim, hidden_layer]  ==> Relu()  (H)   *   [hidden_layer, vocab_size]   ==> Softmax() 
              [1,embedding_dim]                                                                                       [vocab_size]
      x_2
      x_3    
    """
    def __init__(self, vocab_size, embedding_dim, hidden_layer = 128):
      super(CBOW_Batch, self).__init__()
      self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
      self.linear1 = nn.Linear(in_features=embedding_dim, out_features=hidden_layer)
      self.linear2 = nn.Linear(in_features=hidden_layer, out_features=vocab_size)


    def forward(self, inputs):
      """
      Args:
        inputs (torch.Tensor): An input data sensor inputs.shape = (batch, input_dim)
      """
      in_shape = inputs.shape
      #print(in_shape)
      #https://stackoverflow.com/questions/44790670/torch-sum-a-tensor-along-an-axis
      """
      The simplest and best solution is to use torch.sum().
      To sum all elements of a tensor:
        torch.sum(outputs) # gives back a scalar
      To sum over all rows (i.e. for each column):
        torch.sum(outputs, dim=0) # size = [1, ncol]
      To sum over all columns (i.e. for each row):
        torch.sum(outputs, dim=1) # size = [nrow, 1]

      The only change i had to do to handle large batches is to use torch.sum with dim = 1, i.e. sum for each row = input.        
      """
      x_embedded_sum = torch.sum(self.embeddings(inputs), dim=1).view((in_shape[0],-1))
      """
      https://stackoverflow.com/questions/42479902/how-does-the-view-method-work-in-pytorch
      What is the meaning of parameter -1?
      If there is any situation that you don't know how many rows you want but are sure of the number 
      of columns, then you can specify this with a -1. (Note that you can extend this to tensors with 
      more dimensions. Only one of the axis value can be -1). This is a way of telling the library: 
      "give me a tensor that has these many columns and you compute the appropriate number of rows that 
      is necessary to make this happen".      
      """
      out1 = F.relu(self.linear1(x_embedded_sum))
      out2 = self.linear2(out1)
      log_probs = F.log_softmax(out2, dim=1)
      """
      axis : axis along which we want to calculate the sum value. Otherwise, it will consider arr to be 
      flattened(works on all the axis). axis = 0 means along the column and axis = 1 means working along 
      the row.
      """
      return log_probs

    def get_word_emdedding(self, word):
      word = torch.LongTensor([word_to_ix[word]])
      return self.embeddings(word).view(1,-1)      





In [0]:
"""
Hi. I also had a hard time grasping this, especially because of confusion between CrossEntropyLoss and NLLLoss. Not sure if implementation of negative log likelihood loss was ever explained in courses. In short - CrossEntropyLoss = LogSoftmax + NLLLoss. Here is a quick example with NLLLoss implemenation:
https://forums.fast.ai/t/nllloss-implementation/20028
Quick videos:
https://www.youtube.com/watch?v=tRsSi_sqXjI
https://www.youtube.com/watch?v=bLb_Kp5Q9cw
"""
import torch
torch.manual_seed(1)

def NLLLoss(logs, targets):
    out = torch.zeros_like(targets, dtype=torch.float)
    for i in range(len(targets)):
        out[i] = logs[i][targets[i]]
    return -out.sum()/len(out)

x = torch.randn(3, 5)
y = torch.LongTensor([0, 1, 2])
cross_entropy_loss = torch.nn.CrossEntropyLoss()
log_softmax = torch.nn.LogSoftmax(dim=1)
x_log = log_softmax(x)

nll_loss = torch.nn.NLLLoss()
print("Torch CrossEntropyLoss: ", cross_entropy_loss(x, y))
print("Torch NLL loss: ", nll_loss(x_log, y))
print("Custom NLL loss: ", NLLLoss(x_log, y))
# Torch CrossEntropyLoss:  tensor(1.8739)
# Torch NLL loss:  tensor(1.8739)
# Custom NLL loss:  tensor(1.8739)

Torch CrossEntropyLoss:  tensor(1.8739)
Torch NLL loss:  tensor(1.8739)
Custom NLL loss:  tensor(1.8739)


In [0]:
import numpy as np
training_data = cbow_data[cbow_data['split'] == 'train']
#training_data = training_data.head(n=200000)
print(f"\n Inputs = {training_data.count()}")
counter = 0
losses = []
#these are all hyper params, tried different values, 16, 32, 64....
EMBEDDING_DIM = 64
#the hidden layer doesnt seem to have much impact on the loss.
#now trying number of dimensions
model = CBOW_Batch(vocab_size=len(vocab), embedding_dim=EMBEDDING_DIM, hidden_layer = 64)
loss_funtion = nn.NLLLoss()
#increased learning rate from 0.001 to 0.005, much faster loss decrease.
#ideally should use adam or adagrad or something.
#optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use Adam; the optim package contains many other
# optimization algoriths. The first argument to the Adam constructor tells the
# optimizer which Tensors it should update.
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)



 Inputs = Context    20977
Target     20977
split      20977
dtype: int64


In [0]:
DESIRED_BATCH_SIZE = 32
NUM_OF_SPLITS = int(len(training_data)/DESIRED_BATCH_SIZE)
PRINT = int(NUM_OF_SPLITS/10) #print 10 "." for every iteration
#batch_size * NUM_OF_SPLITS = training data size 

print(f"DESIRED_BATCH_SIZE = {DESIRED_BATCH_SIZE}; NUM_OF_SPLITS = {NUM_OF_SPLITS}, PRINT ={PRINT}")

DESIRED_BATCH_SIZE = 32; NUM_OF_SPLITS = 655, PRINT =65


In [0]:
"""
struggled a lot with the tensor creation. at the end of the daty, pytorch tensors work best with 
numpy arrays. Now, we created the context vector as numpy arrays but were dealing wiht dataframes
hence, i had to convert things back to numpy arrays and work with them.
"""
def make_context_vector_numpy(context_ids_numpy):
    return torch.from_numpy(context_ids_numpy)

for epoch in range(256):
  total_loss = 0
  counter = 0
  for batch in np.array_split(training_data, NUM_OF_SPLITS):
    #fill up a numpy array.
    npArrayContext = np.full((batch.shape[0], 4), -1)
    npArrayTgt = np.full(batch.shape[0], -1)
    for index, current_row in batch.iterrows():
      npArrayContext[index % batch.shape[0] ,:] = current_row['Context']
      npArrayTgt[index % batch.shape[0]] = current_row['Target']

    #print(f"\n npArray =>> {npArray}")
    #print(f"\n counter = [{counter}], npArray.shape = {npArray.shape} \n npArray.dtype = {npArray.dtype}")

    #all these things were tried but didnt work.
    #print(f"\n counter = [{counter}], batch.shape = {batch.shape} \n batch['Context'].shape = {batch['Context'].shape} \n batch['Context'].dtype = {batch['Context'].to_numpy().dtype}")
    #Av = np.hstack(A)
    #print(f"\n batch['Context'] =>> {batch['Context']}")
    #print(f"\n batch['Context'].to_numpy() =>> {batch['Context'].to_numpy()}")

    # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
    # into integer indices and wrap them in tensors)    
    context_vector =  make_context_vector_numpy(npArrayContext)
    #print(f"context_vector.shape = {context_vector.shape}")

    model.zero_grad()
    log_probs = model(context_vector)
    loss = loss_funtion(log_probs, torch.from_numpy(npArrayTgt))
    loss.backward()
    optimizer.step()
    # Get the Python number from a 1-element Tensor by calling tensor.item()
    # print(f"\n counter = [{counter}], loss.item() = {loss.item()}")
    # use the argument "end" to specify the end of line string and print with space
    #print(f".{batch.shape[0]}", end = '')
    if(counter % PRINT == 0):
      print(f".", end = '')
    total_loss += loss.item()
    counter += 1
    # if(counter % 1000 == 0):
    #   print(total_loss)
  print(f"\n[{epoch}]:[{counter}]: Loss = {total_loss}")
  losses.append(total_loss)



...........
[0]:[655]: Loss = 5739.956849098206
...........
[1]:[655]: Loss = 5255.197099208832
...........
[2]:[655]: Loss = 4986.073739528656
...........
[3]:[655]: Loss = 4821.721682548523
...........
[4]:[655]: Loss = 4701.594984054565
...........
[5]:[655]: Loss = 4597.776047706604
...........
[6]:[655]: Loss = 4499.502867221832
...........
[7]:[655]: Loss = 4401.886965751648
...........
[8]:[655]: Loss = 4302.182397842407
...........
[9]:[655]: Loss = 4198.63724565506
...........
[10]:[655]: Loss = 4090.0838899612427
...........
[11]:[655]: Loss = 3975.897720813751
...........
[12]:[655]: Loss = 3855.9206318855286
...........
[13]:[655]: Loss = 3730.5613255500793
...........
[14]:[655]: Loss = 3600.7298860549927
...........
[15]:[655]: Loss = 3467.889811038971
...........
[16]:[655]: Loss = 3334.320272922516
...........
[17]:[655]: Loss = 3202.738653421402
...........
[18]:[655]: Loss = 3075.9734489917755
...........
[19]:[655]: Loss = 2956.779105901718
...........
[20]:[655]: Lo

In [0]:
print(losses)

# with open('/content/gdrive/My Drive/filename.txt', 'w') as f:
#    f.write('values')

#/content/drive/My Drive/Public Shared/Project-2-Word-Embeddings/   
torch.save(model.state_dict(), "cbow-model-frankenstein.bin")

!cp "cbow-model-frankenstein.bin" "/content/drive/My Drive/Public Shared/Project-2-Word-Embeddings/cbow-model-frankenstein.bin"

[5739.956849098206, 5255.197099208832, 4986.073739528656, 4821.721682548523, 4701.594984054565, 4597.776047706604, 4499.502867221832, 4401.886965751648, 4302.182397842407, 4198.63724565506, 4090.0838899612427, 3975.897720813751, 3855.9206318855286, 3730.5613255500793, 3600.7298860549927, 3467.889811038971, 3334.320272922516, 3202.738653421402, 3075.9734489917755, 2956.779105901718, 2846.8639874458313, 2747.1121780872345, 2657.3491978645325, 2576.862882375717, 2504.6778576374054, 2439.6434540748596, 2380.7528121471405, 2327.0920927524567, 2277.7649178504944, 2232.201548218727, 2189.696488380432, 2149.9415471553802, 2112.365389108658, 2076.798861503601, 2042.8781349658966, 2010.3917359113693, 1979.212688922882, 1949.1880702972412, 1920.0910111665726, 1891.9238237142563, 1864.5928556919098, 1837.931304693222, 1811.9696539640427, 1786.6252074241638, 1761.8014137744904, 1737.5589761734009, 1713.8146483898163, 1690.4978272914886, 1667.7006131410599, 1645.3036959171295, 1623.3110632896423, 16

In [0]:
def print_words(context, actual_id, predicted_id):
  #print(context)
  for contextId in context:
    print(ix_to_word[contextId], end=' ')
  print(f"; actual = {ix_to_word[actual_id]} , --  predicted = {ix_to_word[predicted_id]}")

#Word prediction
#given a model, tell the most likely word.
test_data = cbow_data[cbow_data['split'] == 'test']
for index, current_row in training_data.head(n=50).iterrows():
  npArrayContext = np.full((1, 4), -1)
  npArrayTgt = np.full(1, -1)
  npArrayContext[0 ,:] = current_row['Context']
  npArrayTgt[0] = current_row['Target']
  context_vector =  make_context_vector_numpy(npArrayContext)

  #print(context_vector.shape)
  log_probs = model(context_vector) #forward
  indices = log_probs.argmax(1)
  #print predicted and actual.
  print_words(current_row['Context'], current_row['Target'], indices.numpy()[0])





governments religions nations earth ; actual = different , --  predicted = different
confessed guilty ascribed justine ; actual = crime , --  predicted = crime
melancholy abode evidence easily ; actual = doubtless , --  predicted = doubtless
involuntarily endeavoured duties regard ; actual = recollect , --  predicted = recollect
one wandring pollutes day ; actual = thought , --  predicted = thought
occupied attending subsistence decreased ; actual = means , --  predicted = means
occupied arranging old man ; actual = cottage , --  predicted = cottage
whole wretched justice suffered ; actual = mockery , --  predicted = mockery
walton letter th march ; actual = archangel , --  predicted = archangel
picture present human must ; actual = peaceful , --  predicted = peaceful
could precipitate icerifts destroy ; actual = one , --  predicted = one
bear live oblivion country ; actual = poverty , --  predicted = poverty
watching wan friendhis eyes ; actual = countenance , --  predicted = countena

In [0]:
#1. closest words
def pretty_print(results):
    """
    Pretty print embedding results.
    """
    for item in results:
        print ("...[%.2f] - %s"%(item[1], item[0]))

def get_closest(target_word, fn_word_to_idx, embeddings, n=5):
    """
    Get the n closest
    words to your word.
    """
    # Calculate distances to all other words
    target_word_embedding = embeddings[fn_word_to_idx[target_word.lower()]]
    distances = []
    for word, index in fn_word_to_idx.items():
        if word == target_word:
            continue
        distances.append((word, torch.dist(target_word_embedding, embeddings[index])))
    
    results = sorted(distances, key=lambda x: x[1])[1:n+2]
    return results


target_words = ['frankenstein', 'monster', 'science', 'sickness', 'lonely', 'happy']

embeddings =  model.embeddings.weight.data

for target_word in target_words: 
    print(f"======={target_word}=======")
    if target_word not in word_to_ix:
        print("Not in vocabulary")
        continue
    pretty_print(get_closest(target_word, word_to_ix, embeddings, n=5))

=======frankenstein=======
...[8.28] - coward
...[8.60] - songs
...[8.66] - editions
...[8.68] - impenetrable
...[8.77] - inevitable
...[8.82] - adieu
=======monster=======
...[7.94] - ephemeral
...[8.02] - considering
...[8.04] - curiosities
...[8.23] - achieve
...[8.27] - elevating
...[8.35] - dissipate
=======science=======
...[10.00] - castles
...[10.04] - belong
...[10.07] - everywhere
...[10.09] - pretence
...[10.16] - beheld
...[10.18] - incalculable
=======sickness=======
...[8.48] - introduced
...[8.56] - gloried
...[8.62] - lifemy
...[8.68] - wrote
...[8.77] - insufficient
...[8.79] - marked
=======lonely=======
...[8.84] - community
...[8.85] - elevating
...[8.96] - archfiend
...[8.97] - interrupted
...[9.17] - go
...[9.21] - conductors
=======happy=======
...[8.77] - devoured
...[8.78] - condemns
...[8.79] - enticing
...[8.79] - classes
...[8.81] - ability
...[8.91] - hero


In [0]:
#2. word analogy
